<h1>Populate remote grid nodes with labeled tensors </h1>
In this notebook, we will populate our grid nodes with labeled data so that it will be used later by people interested in train models.

**NOTE:** At the time of running this notebook, we were running the grid components in background mode.  

Components:
 - Grid Gateway(http://localhost:8080)
 - Grid Node Bob (http://localhost:3000)
 - Grid Node Alice (http://localhost:3001)
 - Grid Node Bill (http://localhost:3002)

This notebook was made based on <a href="https://github.com/OpenMined/PySyft/blob/dev/examples/tutorials/Part%2010%20-%20Federated%20Learning%20with%20Secure%20Aggregation.ipynb">Part 10: Federated Learning with Encrypted Gradient Aggregation</a> tutorial

<h2>Import dependencies</h2>

In [2]:
import grid as gr
import syft as sy
import torch
import pickle
import time

<h2>Setup config</h2>
Init hook, connect with grid nodes, etc...

In [3]:

hook = sy.TorchHook(torch)

# Connect directly to grid nodes
nodes = [("http://localhost:3000/", "Bob"),
         ("http://localhost:3001/", "Alice"),
         ("http://localhost:3002/", "Bill") ]

compute_nodes = []
for i, node in enumerate(nodes):
    compute_nodes.append( gr.WebsocketGridClient(hook, node[0], id=node[1]) )
    compute_nodes[i].connect()

<h2>Load dataset</h2>
Load and prepare the dataset

In [6]:
# Load Data
with open('../dataset/boston_housing.pickle','rb') as f:
    ((X, y), (X_test, y_test)) = pickle.load(f)

X = torch.from_numpy(X).float()
y = torch.from_numpy(y).float()

# preprocessing
mean = X.mean(0, keepdim=True)
dev = X.std(0, keepdim=True)
mean[:, 3] = 0. # the feature at column 3 is binary,
dev[:, 3] = 1.  # so I'd rather not standardize it
X = (X - mean) / dev

<h2> Split dataset </h2>
We will split our dataset to send to nodes

In [7]:
datasets = torch.split(X, int(len(X) / len(compute_nodes)), dim=0 ) #tuple of chunks (dataset / number of nodes)
labels = torch.split(y, int(len(X) / len(compute_nodes)), dim=0 )  #tuple of chunks (labels / number of nodes)

<h2>Tagging tensors</h2>

In [8]:
tag_x = []
tag_y = []

for i in range(len(compute_nodes)):
    tag_x.append(datasets[i].tag("#X", "#boston", "#housing").describe("The input datapoints to the boston housing dataset."))
    tag_y.append(labels[i].tag("#Y", "#boston", "#housing").describe("Boston Housing labels"))

<h2>Sending our tensors to grid nodes</h2>

In [10]:
# NOTE: For some reason, there is strange behavior when trying to send within a loop.
# Ex : tag_x[i].send(compute_nodes[i])
# When resolved, this should be updated.

shared_x1 = tag_x[0].send(compute_nodes[0]) # First chunk of dataset to Bob
shared_x2 = tag_x[1].send(compute_nodes[1]) # Second chunk of dataset to Alice
shared_x3 = tag_x[2].send(compute_nodes[2]) # Third chunk of dataset to Bill

shared_y1 = tag_y[0].send(compute_nodes[0]) # First chunk of labels to Bob
shared_y2 = tag_y[1].send(compute_nodes[1]) # Second chunk of labels to Alice
shared_y3 = tag_y[2].send(compute_nodes[2]) # Third chunk of labels to Bill

In [11]:
print("X tensor pointers: ", shared_x1, shared_x2, shared_x3)
print("Y tensor pointers: ", shared_y1, shared_y2, shared_y3)

X tensor pointers:  (Wrapper)>[PointerTensor | me:52365936731 -> Bob:9775387043]
	Tags: #boston #X #housing 
	Shape: torch.Size([134, 13])
	Description: The input datapoints to the boston housing dataset.... (Wrapper)>[PointerTensor | me:33920212345 -> Alice:52110458330]
	Tags: #boston #X #housing 
	Shape: torch.Size([134, 13])
	Description: The input datapoints to the boston housing dataset.... (Wrapper)>[PointerTensor | me:10949414394 -> Bill:92837193854]
	Tags: #boston #X #housing 
	Shape: torch.Size([134, 13])
	Description: The input datapoints to the boston housing dataset....
Y tensor pointers:  (Wrapper)>[PointerTensor | me:33247328859 -> Bob:84773015288]
	Tags: #boston #housing #Y 
	Shape: torch.Size([134])
	Description: Boston Housing labels... (Wrapper)>[PointerTensor | me:40642775889 -> Alice:92386388957]
	Tags: #boston #housing #Y 
	Shape: torch.Size([134])
	Description: Boston Housing labels... (Wrapper)>[PointerTensor | me:49585669480 -> Bill:75786442740]
	Tags: #boston #

<h2>Disconnect nodes</h2>

In [13]:
for i in range(len(compute_nodes)):
    compute_nodes[i].disconnect()